In [ ]:
import pandas as pd
df = pd.read_csv("dataset/df_data_preprocessed.csv")


In [ ]:
# Compute the average word count
df['Word Count'] = pd.to_numeric(df['Word Count'], errors='coerce')

average_word_count = df['Word Count'].mean()

print(f"Average Word Count: {average_word_count}")


Average Word Count: 63.03200883002207


In [ ]:
!pip3 -q install transformers

In [ ]:
df['response_string'] = df.apply(lambda row: f"""Abstract: {row['Abstract']}
Introduction: {row['Introduction']}
Conclusion: {row['Conclusion']}
Limitation: {row['Limitation']}
Experiment_and_Results: {row['Experiment_and_Results']}
Related_Work: {row['Related_Work']}
Methodology: {row['Methodology']}
Dataset: {row['Dataset']}
""", axis=1)


In [ ]:
!pip3 -q install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
generated_text = []

for i in range(len(df)):
    prompt = """You are an AI trained to analyze scientific research and suggest future directions based on the content of a paper.
    Below, you will find sections from a scientific article including the 'Abstract', 'Introduction', 'Conclusion','Limitation','Experiment and Results','Related Work','Methodology'
    of a scientific paper.
    Based on these details, please generate comprehensive and plausible future work suggestions that could extend the research findings,
    address limitations, and propose new avenues for exploration.
    Generate a future work based on these texts. Future work should be within 100 words. \n""" + df['response_string'][i]
    summary_text = ""  # Initialize an empty string to collect the limitation text

    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        stream=True,
        temperature=0.2  # Adjust the temperature as needed, max_tokens=150
    )
    for chunk in stream:
        summary_chunk = chunk.choices[0].delta.content or ""
        summary_text += summary_chunk

    text_chunks = []
    text_chunks.append(summary_text)
    generated_text.append(text_chunks)


In [ ]:
df_generated_future_work = pd.DataFrame(generated_summary, columns=['Future_Work'])
df['Future_Work'] = df_generated_future_work['Future_Work']

In [ ]:
df.to_csv('dataset/df_gpt_3.csv', index=False)